In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
(dataset, info) = tfds.load("oxford_iiit_pet",
                            with_info=True,
                            as_supervised=True)

train_ds = dataset['train']
test_ds = dataset['test']

class_names = info.features['label'].names

In [ ]:
dog_labels = tf.constant(
    [i for i, name in enumerate(class_names) if name[0].islower()], dtype=tf.int64
)

def is_dog(image, label):
    return tf.reduce_any(tf.equal(dog_labels, label))

train_ds = train_ds.filter(is_dog)
test_ds = test_ds.filter(is_dog)

dog_class_names = [class_names[i] for i in dog_labels.numpy()]
NUM_CLASSES = len(dog_class_names)

In [ ]:
plt.figure(figsize=(10,6))

label_map = {original_idx: new_idx for new_idx, original_idx in enumerate(dog_labels.numpy())}

for i, (image, label) in enumerate(train_ds.take(6)):
    plt.subplot(2,3,i+1)
    plt.imshow(image)
    plt.title(dog_class_names[label_map[label.numpy()]])
    plt.axis('off')
plt.show()

In [ ]:
IMG_SIZE = 224

label_table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(list(label_map.keys()), dtype=tf.int64),
        values=tf.constant(list(label_map.values()), dtype=tf.int64)
    ),
    default_value=-1
)

valid_labels = tf.constant(list(label_map.keys()), dtype=tf.int64)

def filter_valid(image, label):
    return tf.reduce_any(tf.equal(label, valid_labels))

train_raw = dataset['train'].filter(filter_valid)
test_raw = dataset['test'].filter(filter_valid)

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    remapped_label = label_table.lookup(label)
    return image, remapped_label

BATCH_SIZE = 32

train_count = info.splits['train'].num_examples
test_count = info.splits['test'].num_examples

train_ds = train_raw
test_ds = test_raw

train_ds = (
    train_ds
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .shuffle(1000)
    .batch(BATCH_SIZE)
    .repeat()
    .prefetch(tf.data.AUTOTUNE)
)

test_ds = (
    test_ds
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
counter = Counter()

for _, label in dataset['train']:   # ← NOT train_ds
    counter[label.numpy()] += 1

print("Class Distribution:")
for i in range(NUM_CLASSES):
    print(dog_class_names[i], ":", counter[i])

In [ ]:
def create_cnn_model(num_classes):
    model = tf.keras.Sequential([
        # First Convolutional Block
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Dropout(0.25),

        # Second Convolutional Block
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Dropout(0.25),

        # Third Convolutional Block
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Dropout(0.25),

        # Fourth Convolutional Block
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Dropout(0.25),

        # Fifth Convolutional Block
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Dropout(0.25),

        # Global Average Pooling and Dense Layers
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    return model

In [ ]:
# Create and compile the model
model = create_cnn_model(NUM_CLASSES)

# Display model summary
model.summary()

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=60,
    steps_per_epoch=train_count // BATCH_SIZE,
    validation_steps=test_count // BATCH_SIZE
)

In [ ]:
model.save("dog_breed_model.keras")
print("Model saved successfully.")

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title("Loss")
plt.legend()

plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print("Test Accuracy:", test_acc)

In [ ]:
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images, verbose=0)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=dog_class_names))

In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(12,10))
sns.heatmap(cm, cmap='Blues', annot=False, fmt='d')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
def predict_and_display(index):
    image, label = list(test_ds.unbatch())[index]
    input_img = tf.expand_dims(image, axis=0)

    probs = model.predict(input_img, verbose=0)[0]
    pred_class = np.argmax(probs)

    plt.figure(figsize=(12,5))

    plt.subplot(1,2,1)
    plt.imshow(image)
    plt.title(f"True: {dog_class_names[label.numpy()]}")
    plt.axis('off')

    plt.subplot(1,2,2)
    # Show top 5 predictions
    top_5_indices = np.argsort(probs)[-5:][::-1]
    top_5_probs = probs[top_5_indices] * 100
    top_5_names = [dog_class_names[i] for i in top_5_indices]

    colors = ['green' if i == pred_class else 'blue' for i in range(5)]
    plt.barh(range(5), top_5_probs, color=colors)
    plt.yticks(range(5), top_5_names)
    plt.xlabel('Probability (%)')
    plt.title(f"Predicted: {dog_class_names[pred_class]}")

    plt.tight_layout()
    plt.show()

# Test prediction
predict_and_display(10)

In [ ]:
predict_and_display(50)

In [ ]:
predict_and_display(70)